<a href="https://colab.research.google.com/github/Metamorphozis/model_ml/blob/main/Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Импорт библиотек:

Импортируем pandas для работы с DataFrame и numpy для математических операций.

In [1]:
import pandas as pd
import numpy as np

2. Определение класса MyLineReg:

__init__: Конструктор класса, инициализирует атрибуты класса (параметры модели).

In [2]:
class MyLineReg:
    def __init__(self, n_iter=100, learning_rate=0.1, metric=None):
        self.n_iter = n_iter  # Количество итераций обучения
        self.learning_rate = learning_rate  # Скорость обучения
        self.weights = None  # Массив весов модели (инициализируется в `fit`)
        self.metric = metric  # Метрика для оценки модели (MAE, MSE, RMSE, MAPE, R2)
        self.best_score = None  # Лучшее значение метрики, которое мы найдем в процессе обучения

3. Метод fit:

fit: Метод обучения модели. Он принимает на вход матрицу признаков X и вектор целевых значений y.

In [3]:
def fit(self, X, y, verbose=False):
        # Добавляем единичный столбец к X для смещения
       X = pd.concat([pd.Series(np.ones(X.shape[0])), X], axis=1)

       self.weights = np.zeros(X.shape[1])  # Инициализируем веса нулями

       start_loss = float('inf')  # Инициализируем start_loss бесконечностью
       score = None  # Инициализируем score как None

       for i in range(self.n_iter):
           y_pred = np.dot(X, self.weights)  # Делаем предсказание

           loss = ((y - y_pred)**2).mean()  # Считаем среднеквадратичную ошибку

            # Обновляем веса
           gradient = 2 * np.dot(X.T, (y_pred - y)) / X.shape[0]  # Вычисляем градиент
           self.weights -= self.learning_rate * gradient  # Обновляем веса с помощью градиентного спуска

            # Рассчитываем метрику на последней итерации или если verbose=True
           if verbose or i == self.n_iter - 1:
               score = self._calculate_metric(y, y_pred)  # Вычисляем метрику
               if verbose:
                   print(f"{i+1} | loss: {loss:.2f} | {self.metric}: {score:.2f}")

           if loss < start_loss:
              start_loss = loss # Добавили эту строку
              self.best_score = score
           else:
                break # Прерываем цикл, если loss не уменьшается

4. Метод get_coef:

get_coef: Возвращает веса модели (без смещения).

In [4]:
def get_coef(self):
      return self.weights[1:]  # Возвращает веса без смещения

5. Метод predict:

predict: Делает предсказания для новой матрицы признаков X.

In [5]:
def predict(self, X):
        X = pd.concat([pd.Series(np.ones(X.shape[0])), X], axis=1)  # Добавляем единичный столбец
        predictions = np.dot(X, self.weights)  # Делаем предсказание
        return predictions  # Возвращаем вектор предсказаний

6. Метод get_best_score:

get_best_score: Возвращает лучшее значение метрики, которое было достигнуто в процессе обучения.

In [6]:
def get_best_score(self):
        if self.best_score is None:
            return "No score available"  # Возвращаем строку, если best_score не доступен
        else:
            return round(self.best_score, 10)  # Возвращаем округленное значение best_score

7. Метод _calculate_metric:

_calculate_metric: Вычисляет значение метрики в зависимости от параметра self.metric.

In [7]:
def _calculate_metric(self, y_true, y_pred):
    if self.metric == 'mae':
        score = np.abs(y_true - y_pred).mean()
    elif self.metric == 'mse':
         score = ((y_true - y_pred)**2).mean()
    elif self.metric == 'rmse':
         score = np.sqrt(((y_true - y_pred)**2).mean())
    elif self.metric == 'mape':
         score = np.abs((y_true - y_pred) / y_true).mean() * 100
    elif self.metric == 'r2':
        ss_res = np.sum((y_true - y_pred)**2)
        ss_tot = np.sum((y_true - np.mean(y_true))**2)
        score = 1 - (ss_res / ss_tot)
    else:
         raise ValueError("Unsupported metric")

    return score

Пример использования:

In [8]:
# Загрузка данных (пример)
X = pd.DataFrame([[1, 2], [3, 4], [5, 6]])
y = pd.Series([7, 8, 9])

# Создание объекта модели
model = MyLineReg(n_iter=1000, learning_rate=0.01, metric='mae')

# Обучение модели
model.fit(X, y, verbose=True)

# Получение весов
print(model.get_coef())

# Получение лучшего значения метрики
print(model.get_best_score())

# Делаем предсказания для новых данных
new_X = pd.DataFrame([[10, 11]])
predictions = model.predict(new_X)
print(predictions)

AttributeError: 'MyLineReg' object has no attribute 'fit'

In [ ]:
import pandas as pd
import numpy as np

class MyLineReg:
    def __init__(self, n_iter=100, learning_rate=0.1, metric=None):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = None
        self.metric = metric
        self.best_score = None

    def fit(self, X, y, verbose=False):
        # Добавляем единичный столбец к X для смещения
        X = pd.concat([pd.Series(np.ones(X.shape[0])), X], axis=1)

        self.weights = np.zeros(X.shape[1])

        start_loss = float('inf')
        score = None

        for i in range(self.n_iter):
            y_pred = np.dot(X, self.weights)

            loss = ((y - y_pred)**2).mean()

            # Обновляем веса
            gradient = 2 * np.dot(X.T, (y_pred - y)) / X.shape[0]
            self.weights -= self.learning_rate * gradient

            # Рассчитываем метрику на последней итерации или если verbose=True
            if verbose or i == self.n_iter - 1:
                score = self._calculate_metric(y, y_pred)
                if verbose:
                    print(f"{i+1} | loss: {loss:.2f} | {self.metric}: {score:.2f}")

            if loss < start_loss:
                start_loss = loss # Добавили эту строку
                self.best_score = score
            else:
                break

    def get_coef(self):
        return self.weights[1:]

    def predict(self, X):
        X = pd.concat([pd.Series(np.ones(X.shape[0])), X], axis=1)
        predictions = np.dot(X, self.weights)
        return predictions

    def get_best_score(self):
        if self.best_score is None:
            return "No score available"  # Возвращаем строку вместо None
        else:
            return round(self.best_score, 10)  # Округляем значение метрики сразу здесь

    def _calculate_metric(self, y_true, y_pred):
        if self.metric == 'mae':
            score = np.abs(y_true - y_pred).mean()
        elif self.metric == 'mse':
            score = ((y_true - y_pred)**2).mean()
        elif self.metric == 'rmse':
            score = np.sqrt(((y_true - y_pred)**2).mean())
        elif self.metric == 'mape':
            score = np.abs((y_true - y_pred) / y_true).mean() * 100
        elif self.metric == 'r2':
            ss_res = np.sum((y_true - y_pred)**2)
            ss_tot = np.sum((y_true - np.mean(y_true))**2)
            score = 1 - (ss_res / ss_tot)
        else:
            raise ValueError("Unsupported metric")

        return score

In [ ]:
# Загрузка данных (пример)
X = pd.DataFrame([[1, 2], [3, 4], [5, 6]])
y = pd.Series([7, 8, 9])

# Обучение с MSE (без явного указания metric)
model_mse = MyLineReg(n_iter=1000, learning_rate=0.01)  # metric=None по умолчанию
model_mse.fit(X, y)
print(f"MSE: {model_mse.get_best_score()}")